In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [4]:
from elasticsearch import Elasticsearch

es_client=Elasticsearch('http://localhost:9200')

**Q2** INDEXING DATA

In [6]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-question-hw1"

es_client.indices.create(index=index_name, body=index_settings)

/tmp/ipykernel_2089/1514437213.py:18: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es_client.indices.create(index=index_name, body=index_settings)


{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'course-question-hw1'}

In [8]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name,document=doc)

/home/codespace/.python/current/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 0/948 [00:00<?, ?it/s]

100%|██████████| 948/948 [00:04<00:00, 220.47it/s]


**Q3** SEARCHING DATA

In [9]:
query='How do execute a command on a Kubernetes pod?'

In [10]:
search_query={
    "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"], #question has boost of 4
                        "type": "best_fields"
                    }
                },
            }
        }
}

In [12]:
response=es_client.search(index=index_name,body=search_query)

/tmp/ipykernel_2089/95407596.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  response=es_client.search(index=index_name,body=search_query)


In [13]:
response

{'took': 342,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 739, 'relation': 'eq'},
  'max_score': 44.50556,
  'hits': [{'_index': 'course-question-hw1',
    '_id': 'ypPWcpcBdlrUHpTOw0yU',
    '_score': 44.50556,
    '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
     'section': '5. Deploying Machine Learning Models',
     'question': 'How do I debug a docker container?',
     'course': 'machine-learning-zoomcamp'}},
   {'_index': 'course-question-hw1',
    '_id': 'WZPWcpcBdlrUHpTOxU3J',
    '_score': 35.433445,
    '_source': {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke',
     'section': '10. Kubernetes a

**Q4** FILTERING

In [15]:
query1='How do copy a file to a Docker container?'

In [16]:
search_query1={
    "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query1,
                        "fields": ["question^4", "text"], #question has boost of 4
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
}

response=es_client.search(index=index_name,body=search_query1)

response

/tmp/ipykernel_2089/4171248687.py:21: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  response=es_client.search(index=index_name,body=search_query1)


{'took': 17,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 340, 'relation': 'eq'},
  'max_score': 73.38676,
  'hits': [{'_index': 'course-question-hw1',
    '_id': 'ypPWcpcBdlrUHpTOw0yU',
    '_score': 73.38676,
    '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
     'section': '5. Deploying Machine Learning Models',
     'question': 'How do I debug a docker container?',
     'course': 'machine-learning-zoomcamp'}},
   {'_index': 'course-question-hw1',
    '_id': '6ZPWcpcBdlrUHpTOw0z_',
    '_score': 66.688705,
    '_source': {'text': "You can copy files from your local machine into a Docker container using the docker c

**Q5** BUILD A PROMPT

In [18]:
result_docs=[]

for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])
    
result_docs
        

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [ ]:
context_template=""

for doc in result_docs:
    context_template = context_template+f"Q: {doc['question']}\nA: {doc['text']}\n\n"

In [61]:
context_template=""

for doc in result_docs:
    context_template = context_template+f"""
Q: {doc['question']}
A: {doc['text']}
""".strip()+'\n\n'

In [62]:
print(context_template)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [63]:
query1

'How do copy a file to a Docker container?'

In [64]:


prompt_template = f"""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {query1}

CONTEXT:
{context_template}
""".strip()

prompt_template

'You\'re a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\nUse only the facts from the CONTEXT when answering the QUESTION.\n\nQUESTION: How do copy a file to a Docker container?\n\nCONTEXT:\nQ: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container

In [65]:
len(prompt_template)

1446

**Q6** TOKENS

In [68]:
import os
import google.generativeai as genai
from dotenv import load_dotenv

load_dotenv()

genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
model=genai.GenerativeModel("gemini-1.5-pro")

tokens=model.count_tokens(prompt_template)

tokens

total_tokens: 353